In [ ]:
import os
import sys

folder_name = "discovering_modern_cpp"
colab_flag = False
# 判断是否在 Google Colab 环境中运行
if 'google.colab' in sys.modules:
  colab_flag = True
  if not os.path.exists(folder_name):
    !git clone https://github.com/jyellow/discovering_modern_cpp
  else:
    print(f"Folder '{folder_name}' already exists.")
else:
  # 在本地macOS环境
  # TODO: 目前使用GitHub Desktop的默认仓库路径，后续修改为针对Jupyter Notebook的相对路径
  github_desktop_path = os.path.expanduser("~/Documents/GitHub/")
  folder_path = os.path.join(github_desktop_path, folder_name)
  if not os.path.exists(folder_path):
    print(f"Folder '{folder_path}' does not exist. Clone the repository manually in github.")
    !git clone https://github.com/jyellow/discovering_modern_cpp
  else:
    print(f"Folder '{folder_name}' found at '{folder_path}'.")

## 简单的测试一下通过Google Colab运行C++代码的可用性

In [ ]:
if not colab_flag:
  # 本地环境
  %cd ~/Documents/GitHub
%cd discovering_modern_cpp/c++03/

In [ ]:
# %cd ../

In [ ]:
!pwd

In [ ]:
# 查看项目的原始Makefile文件
!cat makefile

In [ ]:
if colab_flag:
  # 在Colab环境中，重命名Makefile为makefile.bak.txt
  print("将当前环境下的Makefile文件备份为makefile.bak.txt")
  !cp makefile makefile.bak.txt
else:
  # 本地环境执行过一次以后就不需要再执行了
  if os.path.isfile("makefile.bak.txt"):
    print("本地环境的原始Makefile文件已被重命名为makefile.bak.txt，不再需要执行重命名操作。")
  else:
    print("将当前环境下的Makefile文件备份为makefile.bak.txt")
    !cp makefile makefile.bak.txt

In [ ]:
%%writefile Makefile

ALLEXE =
#CXX= g++-4.4
#CXX= g++-4.8
#CXX= clang++-3.4
#CXX= mpiCC
# Google Colab环境没有默认安装Boost库
#CXXFLAGS+=  -I${BOOST_ROOT}
#CXXFLAGS+=  -I${MTL}
CXXFLAGS+= -Wall -pedantic
#CXXFLAGS+= -w               # no warnings at all

CXXFLAGS+= -Wall -g -O0
#CXXFLAGS+= -Wall -O3 -DNDEBUG -ffast-math
CXXFLAGS+= -funroll-loops
#CXXFLAGS+= -L/usr/local/lib -lboost_mpi-gcc41-mt -lboost_serialization-gcc41-mt # for boost::mpi
#CXXFLAGS+= -L/home/pgottsch/projects/boost/boost_1_38_0/bin.v2/libs/mpi/build/gcc-4.3.3/release/link-static/threading-multi -lboost_mpi -L/home/pgottsch/projects/boost/boost_1_38_0/bin.v2/libs/serialization/build/gcc-4.3.3/release/link-static/threading-multi -lboost_serialization # for boost::mpi

# For OCCI example
# CXXFLAGS+=  -I/u01/app/oracle/product/11.2.0/xe/rdbms/public
# LOADLIBES+=   -L/u01/app/oracle/product/11.2.0/xe/lib/
# LOADLIBES+=   -locci -lclntsh
#### needs: # export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/u01/app/oracle/product/11.2.0/xe/lib

# 由于本地的macOS不支持静态编译，而Google Colab环境是Linux环境，支持静态编译
# 因此需要通过uname命令判断操作系统类型，从而设置静态链接标志
UNAME_S := $(shell uname -s)


ifeq ($(UNAME_S),Darwin)
    # macOS 不支持静态链接
    STATIC_FLAG =
    # macOS环境下将编译二进制文件的目录设置为上级目录的build文件夹
    # 从而能够统一管理编译生成的二进制文件
    BUILD_DIR = ../build
else
    # Linux 支持静态链接
    STATIC_FLAG = -static
    # Google Colab环境不需要特殊处理编译二进制文件的目录
    BUILD_DIR = .
endif


# 注意：Makefile的构建命令行之前必须使用Tab键缩进，而不是空格键
# 否则会抛出错误：missing separator

static_main:	static_main.cpp static_f.cpp
	g++ static_main.cpp static_f.cpp -o $(BUILD_DIR)/static_main

vector_test:	vector_test.cpp
	g++ vector_test.cpp -o $(BUILD_DIR)/vector_test $(STATIC_FLAG)

clean:
	rm -f $(BUILD_DIR)/static_main $(BUILD_DIR)/vector_test


测试运行Makefile文件中的内容

In [ ]:
!make vector_test
!make static_main

In [ ]:
!../build/vector_test

In [ ]:
!../build/static_main

In [ ]:
!make clean

## C++ 11

In [ ]:
!whereis g++

In [ ]:
# 切换到discovering_modern_cpp/c++11/目录
if colab_flag:
  # 在Google Colab环境中，切换到discovering_modern_cpp/c++11/目录
  %cd /content/
  %cd discovering_modern_cpp/c++11/
else:
  # 在本地环境中
  %cd ~/Documents/GitHub/
  %cd discovering_modern_cpp/c++11/

In [ ]:
!pwd

In [ ]:
print(colab_flag)
if colab_flag:
  # 在Colab环境中，重命名Makefile为makefile.bak.txt
  print("将当前环境下的Makefile文件备份为makefile.bak.txt")
  !cp makefile makefile.bak.txt
else:
  # 本地环境执行过一次以后就不需要再执行了
  if os.path.isfile("makefile.bak.txt"):
    print("本地环境的原始Makefile文件已被重命名为makefile.bak.txt，不再需要执行重命名操作。")
  else:
    print("将当前环境下的Makefile文件备份为makefile.bak.txt")
    !cp makefile makefile.bak.txt

In [ ]:
%%writefile Makefile
BUGGY  = 'variadic_sum_left' # separate by \|
ALLEXE = $(shell ls *.cpp | sed -e 's/\.cpp//'| grep -v ${BUGGY})


# optional flags
#CXXFLAGS+=  -I${BOOST_ROOT}
#CXXFLAGS+= -Wall -pedantic -Wno-unused-variable
# CXXFLAGS+= -m32
 # CXXFLAGS+= -Wno-missing-braces # affects only clang++, ignored by g++
#CXXFLAGS+= -Wno-c++11-narrowing
CXXFLAGS+= -g -O0
#CXXFLAGS+= -O3 -DNDEBUG
#CXXFLAGS+= -ffast-math
#CXXFLAGS+= -funroll-loops
# CXXFLAGS+= -v # really verbose
#CXXFLAGS+= -I/home/pgottsch/projects/boost/boost_1_55_0
CXXFLAGS+= -I${MTL}
#CXXFLAGS+= -I/home/pgottsch/projects/mtl4-branches/pro
CXXFLAGS+= -DMTL_WITH_INITLIST -DMTL_WITH_AUTO -DMTL_WITH_RANGEDFOR -DMTL_WITH_VARIADIC_TEMPLATE
CXXFLAGS+= -DMTL_WITH_DEFAULTIMPL -DMTL_WITH_TEMPLATE_ALIAS -DMTL_WITH_STATICASSERT -DMTL_WITH_MOVE
#CXXFLAGS+= -Wno-unused-local-typedefs

# for the OCCI example
# CXXFLAGS+=  -I/u01/app/oracle/product/11.2.0/xe/rdbms/public
# LOADLIBES+=   -L/u01/app/oracle/product/11.2.0/xe/lib/
# LOADLIBES+=   -locci -lclntsh -lnnz11
#### needs: # export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/u01/app/oracle/product/11.2.0/xe/lib

# turn on C++11
# CXXFLAGS+= -std=c++14 # for quick experiments
CXXFLAGS+= -std=c++11
# CXXFLAGS+= -std=c++0x

# library must be indicated  with clang <= 3.2
# CXXFLAGS+= -stdlib=libc++

#CXX= g++-4.8
CXX=/usr/bin/g++
#CXX= g++-5
#CXX= clang++-3.4

UNAME_S := $(shell uname -s)

ifeq ($(UNAME_S),Darwin)
    # macOS 不支持静态链接
    STATIC_FLAG =
    # macOS环境下将编译二进制文件的目录设置为上级目录的build文件夹
    # 从而能够统一管理编译生成的二进制文件
    BUILD_DIR = ../build
else
    # Linux 支持静态链接
    STATIC_FLAG = -static
    # Google Colab环境不需要特殊处理编译二进制文件的目录
    BUILD_DIR = .
endif

# 给所有可执行文件添加编译后的目录前缀
ALLEXE_WITH_DIR = $(addprefix ${BUILD_DIR}/,${ALLEXE})

# 执行make时，默认执行的目标
%:	%.cpp
	@echo "Compiling $@"
	# 添加指定的二进制生成路径
	# g++ ${CXXFLAGS} -o ${BUILD_DIR}/$@ $? ${LOADLIBES}
	# @ ${BUILD_DIR}/$@
	${CXX} ${CXXFLAGS} -o ${BUILD_DIR}/$@ $? ${LOADLIBES} # -Wno-unused-but-set-variable

# additional flags for binary installation of clang 3.3, shouldn't be needed :-!
# CXXFLAGS+= -I/usr/local/clang+llvm-3.3/lib/c++/v1
# LDFLAGS+= -nodefaultlibs -lstdc++ -lm -lc -lgcc_s -lgcc

# For SDL 1.2
# 加载SDL库
SDL_FLAGS= -D_GNU_SOURCE=1 -D_REENTRANT -lSDL
MTL_FLAGS= -I${MTL}

default: ${ALLEXE}
	 echo "${BUGGY} -- ${ALLEXE}"

clean:
	rm -rf ${ALLEXE_WITH_DIR}

instance1:	instance1.cpp instance2.cpp
	clang++-3.4 ${CXXFLAGS} -o ${BUILD_DIR}/$@ $?

mandelbrot:	mandelbrot.cpp
	${CXX} -O3 -DNDEBUG -std=c++11 -o ${BUILD_DIR}/$@ $? ${SDL_FLAGS}

julia:	julia.cpp
	${CXX} -O3 -DNDEBUG -std=c++11 -o ${BUILD_DIR}/$@ $? ${SDL_FLAGS}

random_timing:	random_timing.cpp
	${CXX} -O3 -DNDEBUG -std=c++11 -o ${BUILD_DIR}/$@ $?

mangling_example: mangling_example.cpp dings.o
	${CXX} -o ${BUILD_DIR}/$@ $?

multiref_example: multiref1.cpp multiref2.cpp multiref.hpp
	${CXX} -o ${BUILD_DIR}/$@ multiref1.cpp multiref2.cpp

fibonacci:  fibonacci.cpp
	g++ -std=c++0x -o ${BUILD_DIR}/$@ $?

# hpp文件不用显示的添加到编译命令中
poly: poly.cpp
	${CXX} -std=c++11 -o ${BUILD_DIR}/$@ $?

# 用于打印Makefile信息的目标
echo_info:
	@echo "所有的编译后的可执行文件: ${ALLEXE}"

In [ ]:
# 测试Makefile中的通配符规则
# 这里使用sed命令将.cpp后缀去掉，得到可执行文件
# !ls *.cpp | sed -e 's/\.cpp//'
# !export BUGGY='variadic_sum_left'
# !echo ${BUGGY}
# !ls *.cpp | sed -e 's/\.cpp//'| grep -v ${BUGGY}

### 尝试编译fibonacci.cpp文件

In [ ]:
# !make
# !make fibonacci
# !make clean > /dev/null 2>&1
# !make echo_info > make_info.txt

In [ ]:
!./fibonacci 15

### poly.cpp

In [ ]:
# with open('poly.cpp', 'r') as f:
#     lines = f.readlines()
#     for line in lines[:10]:
#         print(line.strip())

# lines[4] = ''
# # 写入文本文件内容的代码，通过python代码修改C++源代码
# with open('poly2.cpp', 'w') as f:
#     f.writelines(lines)

In [ ]:
%%writefile poly.cpp
// Bjarne p. 658  

#include <iostream>
#include <typeinfo>
// 添加自定义的头文件路径地址: c++11/print_compiler.hpp
// 这个头文件主要是用来打印当前环境的编译器信息
#include "print_compiler.hpp"

struct Poly {
    virtual void f() {}
};

struct Non_poly {};

struct D1
  : Poly
{};

struct D2
  : Non_poly
{};

void f(Non_poly& npr, Poly& pr)
{
    // 这里的npr和pr都是引用类型
    std::cout << "NPR NAME: "<<typeid(npr).name() << '\n' << std::endl;
    std::cout << "PR NAME: "<<typeid(pr).name() << '\n' << std::endl;
    // 打印npr和pr的类型信息
    // std::cout << "NPR TYPE: "<<typeid(npr).before(typeid(pr)) << '\n';
    // std::cout << "PR TYPE: "<<typeid(pr).before(typeid(npr)) << '\n';
    
}

int main (int argc, char* argv[]) 
{
    // 打印当前编译器的版本信息
    print_compiler();
    D1 d1;
    D2 d2;
    
    f(d2, d1);
    // f(*static_cast<Poly*>(nullptr), *static_cast<Null_poly*>(nullptr));

    return 0 ;
}



In [ ]:
!make poly

In [ ]:
!../build/Poly

尝试更加友好的类型名称显示

In [ ]:
%%writefile poly.cpp
// Bjarne p. 658  

#include <iostream>
#include <typeinfo>
#include <cxxabi.h>
// 添加自定义的头文件路径地址: c++11/print_compiler.hpp
// 这个头文件主要是用来打印当前环境的编译器信息
#include "print_compiler.hpp"

struct Poly {
    virtual void f() {}
};

struct Non_poly {};

struct D1
  : Poly
{};

struct D2
  : Non_poly
{};

void f(Non_poly& npr, Poly& pr)
{
    // 这里的npr和pr都是引用类型
    std::cout << "NPR NAME: "<<typeid(npr).name() << '\n' << std::endl;
    std::cout << "PR NAME: "<<typeid(pr).name() << '\n' << std::endl;
    // 更加友好的类型显示方式    
    std::cout << "使用cxxabi.h进行更加友好的类型显示方式: " << std::endl;
    std::cout << "NPR NAME: " << abi::__cxa_demangle(typeid(npr).name(), 0, 0, 0) << "\n" << std::endl;
    std::cout << "PR NAME: " << abi::__cxa_demangle(typeid(pr).name(), 0, 0, 0) << std::endl;

}

int main (int argc, char* argv[]) 
{
    // 打印当前编译器的版本信息
    print_compiler();
    D1 d1;
    D2 d2;
    
    f(d2, d1);
    // f(*static_cast<Poly*>(nullptr), *static_cast<Null_poly*>(nullptr));

    return 0 ;
}



### 测试一个其他的Makefile文件中的构建内容

In [ ]:
!make julia